<a href="https://colab.research.google.com/github/liao771018/LINE-BOT/blob/main/LINE_BOT%E4%B8%8A%E8%AA%B2%E5%85%A7%E5%AE%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 寫程式的流程

# 安裝套件

# 引用套件

# 建置主程序

# 為主程序製作交互API接口

# 啟動主程序

# python語言最常見的一種http套件，叫做flask

In [ ]:
# 生成實體物件
line_bot_api = LineBotApi("Channel_Access_token")
handler = WebhookHandler("Channel_Secret")


# 啟動主程序的API入口
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print(body)




In [ ]:
# FollowEvents

# 告知handler，如果收到FollowEvent，則做下面的方法處理
@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    
    # 取出消息內User的資料
    user_profile = line_bot_api.get_profile(event.source.user_id)
        
     # 將用戶資訊存在檔案內
    with open("/content/drive/MyDrive/users.txt", "a") as myfile:
        myfile.write(json.dumps(vars(user_profile),sort_keys=True))
        myfile.write('\n')
    
    # 回覆文字消息與圖片消息
    line_bot_api.reply_message(
        event.reply_token,
        [TextSendMessage('安安，你的個資已被我紀錄了')]
    )

In [ ]:
# MessageEvent, message=TextMessage 文字消息

'''

消息素材準備

'''

# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
)
# 回應用戶的消息物件
specific_text_message=TextSendMessage("官方建議，用戶以文字消息觸發特殊功能的時候，要加上特殊符號")
# 回應圖片消息
# 必須是網址，可以是先將圖片上傳到Cloud Storage
analyze_truth_image_message=ImageSendMessage(
      original_content_url='https://i.imgur.com/M8jnn9B.png',
      preview_image_url='https://i.imgur.com/LtCtgg0.jpg'
      )
'''

設計一個字典
    當用戶輸入相應文字消息時，系統會從此挑揀消息

'''

# 根據自定義菜單四張故事線的圖，設定相對應image
template_message_dict = {
  "@123":specific_text_message,
  "@分析的本質": analyze_truth_image_message
}


# 用戶發出文字消息時， 按條件內容, 回傳文字消息
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    
    # 當用戶傳的消息有 @ 符號且不是最後一個， 則回應字典預設的內容
    if(event.message.text.find('@')!= -1):
        line_bot_api.reply_message(
        event.reply_token,
        template_message_dict.get(event.message.text)
        ) # event.mesage.text是用戶傳來的文字
    else:
        line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text="字典內查無此字，請輸入@123或@分析的本質")
        )

In [ ]:
#用戶點擊button後，觸發postback event，對其回傳做相對應處理

@handler.add(PostbackEvent)
def handle_post_message(event):
    user_profile = line_bot_api.get_profile(event.source.user_id)
    if (event.postback.data.find('Data1')== 0):
        with open("user_profile_business.txt", "a") as myfile:
            myfile.write(json.dumps(vars(user_profile),sort_keys=True))
            myfile.write('\n')
            line_bot_api.reply_message(
            event.reply_token,
                TextMessage(
                    text='請稍待，會有專人與您聯繫'
                )
            )
    elif (event.postback.data.find('Data2') == 0):
        with open("user_profile_tutorial.txt", "a") as myfile:
            myfile.write(json.dumps(vars(user_profile),sort_keys=True))
            myfile.write('\n')
            line_bot_api.reply_message(
            event.reply_token,
                TextMessage(
                    text='請稍待，我們會派專家與您聯繫'
                )
            )
    else:
        pass

In [ ]:
'''
Button篇
    設定模板消息，指定其參數細節。

'''


#引入所需要的消息與模板消息
from linebot.models import (
    MessageEvent, TemplateSendMessage , PostbackEvent
)

#引入按鍵模板
from linebot.models.template import(
    ButtonsTemplate
)


'''
alt_text: Line簡覽視窗所出現的說明文字
template: 所使用的模板
ButtonsTemplate: 按鍵模板
    thumbnail_image_url: 展示圖片
    title: 標題
    text: 說明文字
    actions: 模板行為所使用的行為
    data: 觸發postback後用戶回傳值，可以對其做商業邏輯處理

'''
buttons_template_message = TemplateSendMessage(
    alt_text='Buttons template',
    template=ButtonsTemplate(
        title='更多幫助',
        text='請點擊下方按鈕獲得更多幫助',
        actions=[
          {
            "type": "postback",
            "label": "企業，查找商業結合方案",
            "text": "[::text:]尋找BD",
            "data": "Data1"
          },
          {
            "type": "postback",
            "label": "開發者，尋求教學",
            "text": "[::text:]求助專家",
            "data": "Data2"
          }
        ],
  )
)

In [ ]:
'''

若收到圖片消息時，

先回覆用戶文字消息，並從Line上將照片拿回。

'''

from linebot.models import(
    MessageEvent,ImageMessage, TextSendMessage
)

@handler.add(MessageEvent, message=ImageMessage)
def handle_message(event):
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text='Image has Upload'+ ' ' + event.message.id))
    message_content = line_bot_api.get_message_content(event.message.id)
    with open(event.message.id+'.jpg', 'wb') as fd:
        for chunk in message_content.iter_content():
            fd.write(chunk)


'''

影片消息，更改為 VideoMessage
音訊消息，更改為 AudioMessage

'''

In [ ]:
'''

準備QuickReply的Button


'''

# 引入相關套件
from linebot.models import (
    MessageAction, URIAction,
    PostbackAction, DatetimePickerAction,
    CameraAction, CameraRollAction, LocationAction,
    QuickReply, QuickReplyButton
)

# 創建QuickReplyButton 

## 點擊後，以用戶身份發送文字消息
## MessageAction
textQuickReplyButton = QuickReplyButton(
    action=MessageAction(
        label="發送文字消息", 
        text="text2"
    )
)

## 點擊後，彈跳出選擇時間之視窗
## DatetimePickerAction
dateQuickReplyButton = QuickReplyButton(
    action=DatetimePickerAction(
        label="時間選擇",
        data="data3",                       
        mode="date"
    )
)


## 點擊後，開啟相機
## CameraAction
cameraQuickReplyButton = QuickReplyButton(
    action=CameraAction(label="拍照")
)

## 點擊後，切換至照片相簿選擇
cameraRollQRB = QuickReplyButton(
    action=CameraRollAction(label="選擇照片")
)

## 點擊後，跳出地理位置
locationQRB = QuickReplyButton(
    action=LocationAction(label="地理位置")
)

## 點擊後，以Postback事件回應Server 
postbackQRB = QuickReplyButton(
    action=PostbackAction(label="我是PostbackEvent", data="data1")
)

'''

以QuickReply封裝該些QuickReply Button

'''
## 設計QuickReplyButton的List
quickReplyList = QuickReply(
    items = [textQuickReplyButton, dateQuickReplyButton, cameraQuickReplyButton, cameraRollQRB, locationQRB,postbackQRB]
)

'''

製作TextSendMessage，並將剛封裝的QuickReply放入

'''
## 將quickReplyList 塞入TextSendMessage 中 
from linebot.models import (
    TextSendMessage,
)
quick_reply_text_send_message = TextSendMessage(text='發送問題給用戶，請用戶回答', quick_reply=quickReplyList)

In [ ]:
'''
指定推播
'''

line_bot_api.push_message('USER_ID', TextSendMessage(text='Hello World!'))


'''
廣播
'''

import json
json_object_strings = open("/content/drive/MyDrive/users.txt",'r')
json_array = []
user_id_array = []

for json_object_string in json_object_strings:
    json_object = json.loads(json_object_string)
    json_array.append(json_object)

for user_record in json_array:
    user_id_array.append(user_record.get("user_id"))

line_bot_api.multicast(
    user_id_array,
    TextSendMessage(text='Hello World!')
)